# RLlib Sample Application

First, let's make sure that Ray and RLlib are installed…

In [1]:
!pip install ray[rllib]
!pip install ray[debug]
!pip install ray[tune]
!pip install pandas
!pip install requests
!pip install tensorflow

Then we start Ray…

In [2]:
import ray
import ray.rllib.agents.ppo as ppo

ray.shutdown()
ray.init(ignore_reinit_error=True)

2020-03-29 16:13:43,265	INFO resource_spec.py:212 -- Starting Ray with 4.2 GiB memory available for workers and up to 2.12 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-03-29 16:13:43,745	INFO services.py:1078 -- View the Ray dashboard at localhost:8267


{'node_ip_address': '192.168.1.244',
 'redis_address': '192.168.1.244:17827',
 'object_store_address': '/tmp/ray/session_2020-03-29_16-13-43_254301_27684/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-03-29_16-13-43_254301_27684/sockets/raylet',
 'webui_url': 'localhost:8267',
 'session_dir': '/tmp/ray/session_2020-03-29_16-13-43_254301_27684'}

After a successful launch, there should be a log output line that reads something to the effect of `View the Ray dashboard at localhost:8265` in which case open another browser tab for the Ray dashboard at <http://localhost:8265/>

Next we'll train an RLlib policy with the `CartPole-v0` environment, which is a relatively simple and quick example. For more details about this problem, see the tutorial [*Cartpole - Introduction to Reinforcement Learning (DQN - Deep Q-Learning)*](https://towardsdatascience.com/cartpole-introduction-to-reinforcement-learning-ed0eb5b58288) by [Greg Surma](https://twitter.com/GSurma).

In [3]:
config = ppo.DEFAULT_CONFIG.copy()
config["log_level"] = "WARN"

SELECT_ENV = "Ant-v2"

n_iter = 2
reward_history = []

agent = ppo.PPOTrainer(config, env=SELECT_ENV)

for _ in range(n_iter):
    result = agent.train()
    print(result)

    max_reward = result["episode_reward_max"]
    reward_history.append(max_reward)

    file_name = agent.save("/tmp/ppo")
    print(f"\n{file_name}")

2020-03-29 16:13:44,592	INFO trainer.py:420 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
2020-03-29 16:13:44,631	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.


DependencyNotInstalled: No module named 'mujoco_py'. (HINT: you need to install mujoco_py, and also perform the setup instructions here: https://github.com/openai/mujoco-py/.)

In [ ]:
print(reward_history)

The history of `max_reward` shows that this model `200` by the third iteration -- which is good, since the [*solution*](https://gym.openai.com/envs/CartPole-v0/) for `CartPole-v0` is to get an average reward of `195.0` over a hundred consecutive trials.

In [ ]:
! rllib rollout \
    /tmp/ppo/checkpoint_2/checkpoint-2 \
    --config "{\"env\": \"FrozenLake-v0\"}" --run PPO \
    --steps 2000

Now that we've trained a model, we can look at its resulting policy…

In [ ]:
import pprint

policy = agent.get_policy()
model = policy.model

pprint.pprint(model.variables())
pprint.pprint(model.value_function())

In [ ]:
model.base_model.summary()